In [13]:
!pwd
!pip install shapely


/Users/nicowsendagorta/code/kvkirya/pt-ai/notebooks

[notice] A new release of pip available: 22.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


# Imports

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from shapely.geometry import Polygon
import numpy as np


# Loading JSON data

In [2]:
import json

source_folder = '02'


with open('../raw_data/annotations.json') as json_data:
    data = json.load(json_data)


In [3]:
df_images = pd.DataFrame(data["images"])
df_annotations = pd.DataFrame(data["annotations"])


In [4]:
df_images


,license,id,file_name,width,height,hdri_name,hdri_rotation_deg,camera_rotation_deg
0,0,1,00000001.rgb.png,640,480,construction_yard_4k.hdr,262.228967,78.472965
1,0,2,00000002.rgb.png,640,480,construction_yard_4k.hdr,167.524583,67.888454
2,0,3,00000003.rgb.png,640,480,construction_yard_4k.hdr,10.682487,73.478342
3,0,4,00000004.rgb.png,640,480,construction_yard_4k.hdr,27.743320,72.287580
4,0,5,00000005.rgb.png,640,480,construction_yard_4k.hdr,107.101225,68.926178
...,...,...,...,...,...,...,...,...
59573,0,59574,00059574.rgb.png,640,480,empty_warehouse_01_4k.hdr,321.167844,70.070446
59574,0,59575,00059575.rgb.png,640,480,empty_warehouse_01_4k.hdr,124.626978,63.659398
59575,0,59576,00059576.rgb.png,640,480,empty_warehouse_01_4k.hdr,237.013957,48.973143
59576,0,59577,00059577.rgb.png,640,480,empty_warehouse_01_4k.hdr,150.917579,70.161267


In [5]:
df_annotations


,category_id,image_id,id,iscrowd,color,area,segmentation,bbox,num_keypoints,keypoints,...,pose_category,exercise_category,cuboid_points,zaxis_rotation_deg,attire_top,attire_bottom,presenting_gender,waist_circumference,height,betas
0,0,1,1,0,"[0.09803921568627451, 0.09803921568627451, 0.4...",3075.7804,"[[493.0, 240.99, 485.01, 236.0, 484.0, 227.99,...","[439.01, 187.01, 56.98000000000002, 53.9800000...",14,"[0, 0, 0, 469, 193, 2, 465, 193, 2, 0, 0, 0, 0...",...,situp,[HIIT],"[[438, 186], [484, 186], [502, 194], [451, 194...",15.061740,raglanshirt,track,female,0.730767,1.649463,"[-0.03959597647190094, -0.5644581913948059, 0...."
1,0,1,2,0,"[0.8274509803921568, 0.8666666666666667, 0.894...",2857.8604,"[[151.0, 239.99, 148.0, 239.99, 146.01, 238.0,...","[142.01, 185.01, 51.98000000000002, 54.9800000...",14,"[0, 0, 0, 167, 188, 2, 163, 188, 2, 0, 0, 0, 0...",...,situp,[HIIT],"[[143, 184], [194, 184], [181, 191], [124, 191...",0.801464,raglanshirt,tights,male,0.852622,1.753723,"[0.7923352718353271, -1.2788528203964233, 1.24..."
2,0,1,3,0,"[0.5333333333333333, 1.0, 0.8]",6313.9804,"[[327.0, 258.99, 321.0, 258.99, 319.01, 257.0,...","[302.01, 97.01, 38.98000000000002, 161.9800000...",14,"[0, 0, 0, 324, 116, 2, 321, 116, 2, 0, 0, 0, 0...",...,tree,[Yoga],"[[301, 93], [340, 93], [342, 91], [300, 91], [...",-70.690204,tshirt,jeans,male,0.990933,1.766118,"[0.33408355712890625, 0.24142998456954956, -0...."
3,0,2,4,0,"[0.09803921568627451, 0.09803921568627451, 0.4...",10207.9404,"[[486.0, 185.99, 484.01, 182.0, 486.0, 180.01,...","[477.01, 113.01, 110.98000000000002, 91.98]",14,"[0, 0, 0, 485, 128, 2, 483, 132, 2, 0, 0, 0, 0...",...,sideplank,"[Yoga, HIIT]","[[478, 109], [580, 109], [613, 124], [497, 124...",-32.316700,raglanshirt,track,female,1.104884,1.744472,"[1.1612275838851929, 0.8204484581947327, -1.58..."
4,0,2,5,0,"[0.13333333333333333, 0.3333333333333333, 0.46...",23513.2404,"[[548.0, 338.99, 543.0, 338.99, 541.0, 336.99,...","[506.01, 99.01, 97.98000000000002, 239.9800000...",14,"[0, 0, 0, 577, 120, 2, 567, 118, 2, 0, 0, 0, 0...",...,curls,[HIIT],"[[512, 95], [603, 95], [632, 106], [531, 106],...",-32.057216,tshirt,jeans,male,0.935389,1.854919,"[-0.7777438163757324, -0.040555573999881744, -..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167939,0,59577,167940,0,"[0.8274509803921568, 0.8666666666666667, 0.894...",8632.1004,"[[494.0, 296.99, 487.0, 294.99, 484.0, 291.99,...","[398.01, 229.01, 126.98000000000002, 67.980000...",14,"[0, 0, 0, 410, 263, 2, 411, 265, 2, 0, 0, 0, 0...",...,hipbridge,"[Yoga, HIIT]","[[392, 223], [506, 218], [552, 251], [419, 257...",78.247364,tshirt,track,female,0.951058,1.687193,"[0.6385455131530762, 0.829331636428833, 0.9691..."
167940,0,59578,167941,0,"[0.09803921568627451, 0.09803921568627451, 0.4...",32710.4204,"[[453.0, 354.99, 447.0, 351.99, 440.0, 351.99,...","[249.01, 222.01, 245.98000000000002, 132.98000...",14,"[0, 0, 0, 484, 250, 2, 482, 252, 2, 0, 0, 0, 0...",...,plank,"[Yoga, HIIT]","[[255, 202], [466, 203], [505, 229], [232, 227...",65.123053,raglanshirt,shorts,male,1.046934,1.855765,"[-0.7987443804740906, 0.29380616545677185, 0.8..."
167941,0,59578,167942,0,"[0.13333333333333333, 0.3333333333333333, 0.46...",5325.0804,"[[319.0, 210.99, 316.0, 210.99, 307.0, 205.99,...","[290.01, 137.01, 71.98000000000002, 73.9800000...",14,"[0, 0, 0, 297, 193, 2, 296, 193, 2, 0, 0, 0, 0...",...,legraise,[HIIT],"[[293, 136], [360, 136], [364, 139], [286, 139...",76.773370,tank,shorts,male,0.893962,1.794887,"[0.008171923458576202, 0.0021372123155742884, ..."
167942,0,59578,167943,0,"[0.6, 0.0, 0.4]",5239.8804,"[[332.0, 249.99, 323.0, 247.99, 317.0, 241.99,...","[297.01, 201.01, 106.98000000000002, 48.980000...",14,"[0, 0, 0, 308, 229, 2, 306, 230, 2, 0, 0, 0, 0...",...,hipbridge,"[Yoga, HIIT]","[[298, 198], [413, 199], [420, 206], [294, 206...",78.301114,tank,shorts,male,0.818586,1.795573,"[0.2891143262386322, -1.4047441482543945, 0.37..."


In [6]:
df_annotations.drop(columns = "id", axis = 0, inplace = True)
df_annotations.rename(columns = {"image_id" : "id"}, inplace = True)
len(df_annotations)


167944

In [7]:
#dataset with every single different character in it
df_complete = pd.merge(df_annotations, df_images, on = "id")
df_complete = df_complete[['id', 'file_name', 'pose_variation',
       'pose_category', 'exercise_category', 'bbox', "camera_rotation_deg"]]
df_complete


,id,file_name,pose_variation,pose_category,exercise_category,bbox,camera_rotation_deg
0,1,00000001.rgb.png,situp,situp,[HIIT],"[439.01, 187.01, 56.98000000000002, 53.9800000...",78.472965
1,1,00000001.rgb.png,situp,situp,[HIIT],"[142.01, 185.01, 51.98000000000002, 54.9800000...",78.472965
2,1,00000001.rgb.png,tree_right,tree,[Yoga],"[302.01, 97.01, 38.98000000000002, 161.9800000...",78.472965
3,2,00000002.rgb.png,sideplank_right,sideplank,"[Yoga, HIIT]","[477.01, 113.01, 110.98000000000002, 91.98]",67.888454
4,2,00000002.rgb.png,curls_right,curls,[HIIT],"[506.01, 99.01, 97.98000000000002, 239.9800000...",67.888454
...,...,...,...,...,...,...,...
167939,59577,00059577.rgb.png,hipbridge,hipbridge,"[Yoga, HIIT]","[398.01, 229.01, 126.98000000000002, 67.980000...",70.161267
167940,59578,00059578.rgb.png,plank,plank,"[Yoga, HIIT]","[249.01, 222.01, 245.98000000000002, 132.98000...",78.559729
167941,59578,00059578.rgb.png,legraise,legraise,[HIIT],"[290.01, 137.01, 71.98000000000002, 73.9800000...",78.559729
167942,59578,00059578.rgb.png,hipbridge,hipbridge,"[Yoga, HIIT]","[297.01, 201.01, 106.98000000000002, 48.980000...",78.559729


In [8]:
#so now we need to check wether overlaps
#the bounding boxes are in COCO format so [x_min, y_min, width, height]

#convert coco to pascal_voc so to [x_min, y_min, x_max, y_max]
def coco_to_pascal(X):
    return [X[0], X[1], X[0] + X[2], X[1] + X[3]]


In [9]:
coco_to_pascal([439.01, 187.01, 56.98000000000002, 53.98000000000002])


[439.01, 187.01, 495.99, 240.99]

In [10]:
df_complete["bbox_pascal"] = df_complete["bbox"].apply(coco_to_pascal)


In [11]:
df_complete


,id,file_name,pose_variation,pose_category,exercise_category,bbox,camera_rotation_deg,bbox_pascal
0,1,00000001.rgb.png,situp,situp,[HIIT],"[439.01, 187.01, 56.98000000000002, 53.9800000...",78.472965,"[439.01, 187.01, 495.99, 240.99]"
1,1,00000001.rgb.png,situp,situp,[HIIT],"[142.01, 185.01, 51.98000000000002, 54.9800000...",78.472965,"[142.01, 185.01, 193.99, 239.99]"
2,1,00000001.rgb.png,tree_right,tree,[Yoga],"[302.01, 97.01, 38.98000000000002, 161.9800000...",78.472965,"[302.01, 97.01, 340.99, 258.99]"
3,2,00000002.rgb.png,sideplank_right,sideplank,"[Yoga, HIIT]","[477.01, 113.01, 110.98000000000002, 91.98]",67.888454,"[477.01, 113.01, 587.99, 204.99]"
4,2,00000002.rgb.png,curls_right,curls,[HIIT],"[506.01, 99.01, 97.98000000000002, 239.9800000...",67.888454,"[506.01, 99.01, 603.99, 338.99]"
...,...,...,...,...,...,...,...,...
167939,59577,00059577.rgb.png,hipbridge,hipbridge,"[Yoga, HIIT]","[398.01, 229.01, 126.98000000000002, 67.980000...",70.161267,"[398.01, 229.01, 524.99, 296.99]"
167940,59578,00059578.rgb.png,plank,plank,"[Yoga, HIIT]","[249.01, 222.01, 245.98000000000002, 132.98000...",78.559729,"[249.01, 222.01, 494.99, 354.99]"
167941,59578,00059578.rgb.png,legraise,legraise,[HIIT],"[290.01, 137.01, 71.98000000000002, 73.9800000...",78.559729,"[290.01, 137.01, 361.99, 210.99]"
167942,59578,00059578.rgb.png,hipbridge,hipbridge,"[Yoga, HIIT]","[297.01, 201.01, 106.98000000000002, 48.980000...",78.559729,"[297.01, 201.01, 403.99, 249.99]"


In [12]:
rect1=[0  ,0 ,3,  3]
rect2=[0  ,0 ,3,  3]

def overlap(rect1,rect2):
    p1 = Polygon([(rect1[0],rect1[1]), (rect1[1],rect1[1]),(rect1[2],rect1[3]),(rect1[2],rect1[1])])
    p2 = Polygon([(rect2[0],rect2[1]), (rect2[1],rect2[1]),(rect2[2],rect2[3]),(rect2[2],rect2[1])])
    if not p1.is_valid or not p2.is_valid:
        return False
    return(p1.intersects(p2))


print(overlap(rect1,rect2))


True


In [13]:
per_id = df_complete[df_complete["id"] == 1]
len(per_id)


3

In [14]:
per_id


,id,file_name,pose_variation,pose_category,exercise_category,bbox,camera_rotation_deg,bbox_pascal
0,1,00000001.rgb.png,situp,situp,[HIIT],"[439.01, 187.01, 56.98000000000002, 53.9800000...",78.472965,"[439.01, 187.01, 495.99, 240.99]"
1,1,00000001.rgb.png,situp,situp,[HIIT],"[142.01, 185.01, 51.98000000000002, 54.9800000...",78.472965,"[142.01, 185.01, 193.99, 239.99]"
2,1,00000001.rgb.png,tree_right,tree,[Yoga],"[302.01, 97.01, 38.98000000000002, 161.9800000...",78.472965,"[302.01, 97.01, 340.99, 258.99]"


In [15]:
#function to check for overlaps
def overlap_check_test(X):
    bbox_list = []
    overlap_list = []
    for index, row in X.iterrows():
        bbox_list.append(np.array(row['bbox_pascal']))

    for i in range(len(bbox_list)):
        new_list = []
        for j in range(len(bbox_list)):
            if all(bbox_list[i] == bbox_list[j]) == True:
                new_list.append(False)
            else:
                new_list.append(overlap(bbox_list[i], bbox_list[j]))

        overlap_list.append(new_list)

    overlap_list_final = []

    for lst in overlap_list:
        if any(lst) == True:
            overlap_list_final.append(True)
        if any(lst) == False:
            overlap_list_final.append(False)

    return overlap_list_final


In [16]:
overlap_check_test(per_id)


[True, False, True]

In [17]:
df_complete["overlap"] = df_complete["id"]
df_complete


,id,file_name,pose_variation,pose_category,exercise_category,bbox,camera_rotation_deg,bbox_pascal,overlap
0,1,00000001.rgb.png,situp,situp,[HIIT],"[439.01, 187.01, 56.98000000000002, 53.9800000...",78.472965,"[439.01, 187.01, 495.99, 240.99]",1
1,1,00000001.rgb.png,situp,situp,[HIIT],"[142.01, 185.01, 51.98000000000002, 54.9800000...",78.472965,"[142.01, 185.01, 193.99, 239.99]",1
2,1,00000001.rgb.png,tree_right,tree,[Yoga],"[302.01, 97.01, 38.98000000000002, 161.9800000...",78.472965,"[302.01, 97.01, 340.99, 258.99]",1
3,2,00000002.rgb.png,sideplank_right,sideplank,"[Yoga, HIIT]","[477.01, 113.01, 110.98000000000002, 91.98]",67.888454,"[477.01, 113.01, 587.99, 204.99]",2
4,2,00000002.rgb.png,curls_right,curls,[HIIT],"[506.01, 99.01, 97.98000000000002, 239.9800000...",67.888454,"[506.01, 99.01, 603.99, 338.99]",2
...,...,...,...,...,...,...,...,...,...
167939,59577,00059577.rgb.png,hipbridge,hipbridge,"[Yoga, HIIT]","[398.01, 229.01, 126.98000000000002, 67.980000...",70.161267,"[398.01, 229.01, 524.99, 296.99]",59577
167940,59578,00059578.rgb.png,plank,plank,"[Yoga, HIIT]","[249.01, 222.01, 245.98000000000002, 132.98000...",78.559729,"[249.01, 222.01, 494.99, 354.99]",59578
167941,59578,00059578.rgb.png,legraise,legraise,[HIIT],"[290.01, 137.01, 71.98000000000002, 73.9800000...",78.559729,"[290.01, 137.01, 361.99, 210.99]",59578
167942,59578,00059578.rgb.png,hipbridge,hipbridge,"[Yoga, HIIT]","[297.01, 201.01, 106.98000000000002, 48.980000...",78.559729,"[297.01, 201.01, 403.99, 249.99]",59578


In [18]:
#so lets define the final function
def overlap_check(X):
    bbox_list = []
    overlap_list = []
    for index, row in X.iterrows():
        bbox_list.append(np.array(row['bbox_pascal']))

    for i in range(len(bbox_list)):
        new_list = []
        for j in range(len(bbox_list)):
            if all(bbox_list[i] == bbox_list[j]) == True:
                new_list.append(False)
            else:
                new_list.append(overlap(bbox_list[i], bbox_list[j]))

        overlap_list.append(new_list)

    overlap_list_final = []

    for lst in overlap_list:
        if any(lst) == True:
            overlap_list_final.append(True)
        if any(lst) == False:
            overlap_list_final.append(False)

    for i in range(len(per_id.index)):
        df_complete["overlap"][per_id.index[i]] = overlap_list_final[i]


In [19]:
id_list = list(set(df_complete["id"].to_list()))
id_list


[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185

In [20]:
for i in id_list:
    per_id = df_complete[df_complete["id"] == i]
    overlap_check(per_id)


/tmp/ipykernel_64878/2801952101.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_complete["overlap"][per_id.index[i]] = overlap_list_final[i]
/tmp/ipykernel_64878/2801952101.py:27: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'True' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_complete["overlap"][per_id.index[i]] = overlap_list_final[i]


In [21]:
df_complete


,id,file_name,pose_variation,pose_category,exercise_category,bbox,camera_rotation_deg,bbox_pascal,overlap
0,1,00000001.rgb.png,situp,situp,[HIIT],"[439.01, 187.01, 56.98000000000002, 53.9800000...",78.472965,"[439.01, 187.01, 495.99, 240.99]",True
1,1,00000001.rgb.png,situp,situp,[HIIT],"[142.01, 185.01, 51.98000000000002, 54.9800000...",78.472965,"[142.01, 185.01, 193.99, 239.99]",False
2,1,00000001.rgb.png,tree_right,tree,[Yoga],"[302.01, 97.01, 38.98000000000002, 161.9800000...",78.472965,"[302.01, 97.01, 340.99, 258.99]",True
3,2,00000002.rgb.png,sideplank_right,sideplank,"[Yoga, HIIT]","[477.01, 113.01, 110.98000000000002, 91.98]",67.888454,"[477.01, 113.01, 587.99, 204.99]",True
4,2,00000002.rgb.png,curls_right,curls,[HIIT],"[506.01, 99.01, 97.98000000000002, 239.9800000...",67.888454,"[506.01, 99.01, 603.99, 338.99]",True
...,...,...,...,...,...,...,...,...,...
167939,59577,00059577.rgb.png,hipbridge,hipbridge,"[Yoga, HIIT]","[398.01, 229.01, 126.98000000000002, 67.980000...",70.161267,"[398.01, 229.01, 524.99, 296.99]",True
167940,59578,00059578.rgb.png,plank,plank,"[Yoga, HIIT]","[249.01, 222.01, 245.98000000000002, 132.98000...",78.559729,"[249.01, 222.01, 494.99, 354.99]",True
167941,59578,00059578.rgb.png,legraise,legraise,[HIIT],"[290.01, 137.01, 71.98000000000002, 73.9800000...",78.559729,"[290.01, 137.01, 361.99, 210.99]",True
167942,59578,00059578.rgb.png,hipbridge,hipbridge,"[Yoga, HIIT]","[297.01, 201.01, 106.98000000000002, 48.980000...",78.559729,"[297.01, 201.01, 403.99, 249.99]",True


In [22]:
df_complete_filtered = df_complete[(df_complete["pose_variation"] == "squats") |
                                   (df_complete["pose_variation"] == "pushups") |
                                   (df_complete["pose_variation"] == "lunge_left") |
                                   (df_complete["pose_variation"] == "lunge_right")]


In [23]:
df_complete_filtered


,id,file_name,pose_variation,pose_category,exercise_category,bbox,camera_rotation_deg,bbox_pascal,overlap
7,3,00000003.rgb.png,squats,squats,"[Yoga, HIIT]","[237.01, 96.01, 34.98000000000002, 90.98]",73.478342,"[237.01, 96.01, 271.99, 186.99]",True
10,4,00000004.rgb.png,pushups,pushups,[HIIT],"[0, 246.01, 146.98000000000002, 71.98000000000...",72.287580,"[0, 246.01, 146.98000000000002, 317.99]",False
18,8,00000008.rgb.png,squats,squats,"[Yoga, HIIT]","[300.01, 160.01, 32.98000000000002, 80.9800000...",80.801787,"[300.01, 160.01, 332.99, 240.99]",False
25,10,00000010.rgb.png,pushups,pushups,[HIIT],"[127.00999999999999, 254.01, 166.9800000000000...",68.929825,"[127.00999999999999, 254.01, 293.99, 391.99]",False
26,10,00000010.rgb.png,squats,squats,"[Yoga, HIIT]","[332.01, 86.01, 69.98000000000002, 139.9800000...",68.929825,"[332.01, 86.01, 401.99, 225.99]",True
...,...,...,...,...,...,...,...,...,...
167923,59572,00059572.rgb.png,lunge_left,lunge,"[Yoga, HIIT]","[189.01, 90.01, 102.98000000000002, 132.980000...",67.193017,"[189.01, 90.01, 291.99, 222.99]",True
167928,59573,00059573.rgb.png,pushups,pushups,[HIIT],"[413.01, 308.01, 226.98000000000002, 152.98000...",75.490290,"[413.01, 308.01, 639.99, 460.99]",False
167929,59574,00059574.rgb.png,pushups,pushups,[HIIT],"[166.01, 209.01, 176.98000000000002, 100.98000...",70.070446,"[166.01, 209.01, 342.99, 309.99]",False
167932,59575,00059575.rgb.png,squats,squats,"[Yoga, HIIT]","[170.01, 33.01, 72.98000000000002, 166.9800000...",63.659398,"[170.01, 33.01, 242.99, 199.99]",True


In [24]:
df_complete_filtered = df_complete_filtered[df_complete_filtered['overlap'] == False]
df_complete_filtered


,id,file_name,pose_variation,pose_category,exercise_category,bbox,camera_rotation_deg,bbox_pascal,overlap
10,4,00000004.rgb.png,pushups,pushups,[HIIT],"[0, 246.01, 146.98000000000002, 71.98000000000...",72.287580,"[0, 246.01, 146.98000000000002, 317.99]",False
18,8,00000008.rgb.png,squats,squats,"[Yoga, HIIT]","[300.01, 160.01, 32.98000000000002, 80.9800000...",80.801787,"[300.01, 160.01, 332.99, 240.99]",False
25,10,00000010.rgb.png,pushups,pushups,[HIIT],"[127.00999999999999, 254.01, 166.9800000000000...",68.929825,"[127.00999999999999, 254.01, 293.99, 391.99]",False
28,11,00000011.rgb.png,lunge_left,lunge,"[Yoga, HIIT]","[77.01, 141.01, 103.98, 105.98000000000002]",79.319615,"[77.01, 141.01, 180.99, 246.99]",False
30,12,00000012.rgb.png,pushups,pushups,[HIIT],"[346.01, 199.01, 120.98000000000002, 51.980000...",76.389751,"[346.01, 199.01, 466.99, 250.99]",False
...,...,...,...,...,...,...,...,...,...
167906,59565,00059565.rgb.png,pushups,pushups,[HIIT],"[272.01, 216.01, 103.98000000000002, 29.980000...",74.778263,"[272.01, 216.01, 375.99, 245.99]",False
167918,59570,00059570.rgb.png,pushups,pushups,[HIIT],"[289.01, 196.01, 88.98000000000002, 30.9800000...",76.112042,"[289.01, 196.01, 377.99, 226.99]",False
167919,59571,00059571.rgb.png,pushups,pushups,[HIIT],"[275.01, 318.01, 212.98000000000002, 161.98000...",73.997272,"[275.01, 318.01, 487.99, 479.99]",False
167928,59573,00059573.rgb.png,pushups,pushups,[HIIT],"[413.01, 308.01, 226.98000000000002, 152.98000...",75.490290,"[413.01, 308.01, 639.99, 460.99]",False


In [25]:
# df_complete_filtered['file_name'][image_name]


In [26]:
import os
import shutil
filename = df_complete_filtered['file_name']
filenames =filename.to_list()

pose_variations = df_complete_filtered['pose_variation']


clean_files = []

for file, pose_variation in zip(filenames, pose_variations):
    if file.endswith('.rgb.png'):
        clean_files.append((file, pose_variation))


In [28]:



folder_main = "../raw_data/processed_02"
folder_squats = (f'{folder_main}/squats')
folder_pushups = (f'{folder_main}/pushups')
folder_lunge_left = (f'{folder_main}/lunge_left')
folder_lunge_right = (f'{folder_main}/lunge_right')


os.makedirs(folder_main)
os.makedirs(folder_squats)
os.makedirs(folder_pushups)
os.makedirs(folder_lunge_left)
os.makedirs(folder_lunge_right)

print(f"Folder '{folder_main}' made, good job mate.")


Folder '../raw_data/processed_03' made, good job mate.


[('00000004.rgb.png', 'pushups'),
 ('00000008.rgb.png', 'squats'),
 ('00000010.rgb.png', 'pushups'),
 ('00000011.rgb.png', 'lunge_left'),
 ('00000012.rgb.png', 'pushups'),
 ('00000017.rgb.png', 'pushups'),
 ('00000022.rgb.png', 'pushups'),
 ('00000024.rgb.png', 'pushups'),
 ('00000024.rgb.png', 'squats'),
 ('00000025.rgb.png', 'squats'),
 ('00000026.rgb.png', 'squats'),
 ('00000037.rgb.png', 'pushups'),
 ('00000047.rgb.png', 'pushups'),
 ('00000049.rgb.png', 'squats'),
 ('00000052.rgb.png', 'squats'),
 ('00000052.rgb.png', 'pushups'),
 ('00000058.rgb.png', 'pushups'),
 ('00000058.rgb.png', 'pushups'),
 ('00000061.rgb.png', 'pushups'),
 ('00000061.rgb.png', 'pushups'),
 ('00000064.rgb.png', 'pushups'),
 ('00000071.rgb.png', 'lunge_left'),
 ('00000076.rgb.png', 'squats'),
 ('00000078.rgb.png', 'lunge_left'),
 ('00000084.rgb.png', 'pushups'),
 ('00000085.rgb.png', 'pushups'),
 ('00000091.rgb.png', 'pushups'),
 ('00000093.rgb.png', 'pushups'),
 ('00000096.rgb.png', 'pushups'),
 ('00000102.

In [30]:
for image_name, pose_variation in clean_files:

    source_path = os.path.join(source_folder, image_name)
    new_filename = f"{os.path.splitext(image_name)[0]}_{pose_variation}.rgb.png"
    if pose_variation == 'pushups':
        destination_path = os.path.join(folder_pushups, new_filename)

    if pose_variation == 'squats':
        destination_path = os.path.join(folder_squats, new_filename)

    if pose_variation == 'lunge_left':
        destination_path = os.path.join(folder_lunge_left, new_filename)

    if pose_variation == 'lunge_right':
        destination_path = os.path.join(folder_lunge_right, new_filename)
    try:
        shutil.copy2(source_path, destination_path)
        print(f"File '{image_name}' with pose variation '{pose_variation}' copied to '{folder_main}'")

    except FileNotFoundError:
        print(f"File is not there mate: '{image_name}'")

# example name of file after everything: 00020016.rgb_squats.rgb.png


File is not there mate: '00000004.rgb.png'
File is not there mate: '00000008.rgb.png'
File is not there mate: '00000010.rgb.png'
File is not there mate: '00000011.rgb.png'
File is not there mate: '00000012.rgb.png'
File is not there mate: '00000017.rgb.png'
File is not there mate: '00000022.rgb.png'
File is not there mate: '00000024.rgb.png'
File is not there mate: '00000024.rgb.png'
File is not there mate: '00000025.rgb.png'
File is not there mate: '00000026.rgb.png'
File is not there mate: '00000037.rgb.png'
File is not there mate: '00000047.rgb.png'
File is not there mate: '00000049.rgb.png'
File is not there mate: '00000052.rgb.png'
File is not there mate: '00000052.rgb.png'
File is not there mate: '00000058.rgb.png'
File is not there mate: '00000058.rgb.png'
File is not there mate: '00000061.rgb.png'
File is not there mate: '00000061.rgb.png'
File is not there mate: '00000064.rgb.png'
File is not there mate: '00000071.rgb.png'
File is not there mate: '00000076.rgb.png'
File is not

In [44]:
import tensorflow as tf




batch_size = 32
image_size = (224, 224)


In [45]:


train_dataset = tf.keras.utils.image_dataset_from_directory(
    folder_main,
    labels="inferred",
    label_mode="categorical",  #this is actually done through one-hot encoding
    image_size=image_size,
    batch_size=batch_size,
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=False,
)


Found 0 files belonging to 4 classes.


ValueError: No images found in directory ../raw_data/processed_02. Allowed formats: ('.bmp', '.gif', '.jpeg', '.jpg', '.png')

In [ ]:
# Print class names (based on subdirectory names)
class_names = train_dataset.class_names
print("sick class bro:", class_names)


sick class bro: ['lunge_left', 'lunge_right', 'pushups', 'squats']
